In [521]:
# importing required libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from time import sleep 

In [522]:
# initialising the selenium driver
driver = webdriver.Chrome('chromedriver.exe')


In [523]:
# List of variables
Rank = []
Movie_Name=[]
Release_Year=[]
Maturity_Rating=[]
Runtime = []
Genre= []
Imdb_Rating= []
Metascore= []
Vote_Count= []
Link =[]
Director=[]
Lead_Cast=[]
User_Review=[]
Critic_Review=[]
Country=[]
Language=[]
Budget=[]
Gross_USA=[]
Total_Gross=[]

In [526]:
# list to retrieve multiple pages 
pages = np.arange(101,5101, 100)

# Loop to ittrate over multiple pages 
for page in pages:
    driver.get('https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&adult=include&sort=user_rating,desc&count=100&start={}&ref_=adv_nxt'.format(page))
    sleep(np.random.randint(4,7,1))
    
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    # Loop to scrape data from indivisual page
    for a in soup.findAll('div', attrs = {'class': 'lister-item-content'}):
        
        # html block where, rank, movie name, release year and the link for the inner page is present 
        b = a.find('h3', {'class':'lister-item-header'})
        
        # Rank
        rank = b.find('span', attrs = {'class': 'lister-item-index unbold text-primary'})
        Rank.append(rank.text[0:-1] if rank else 'No data')
        
        # Movie names
        movie_name = b.find('a')
        Movie_Name.append(movie_name.text)
        
        # Release year of the movies
        release_year = b.find('span', attrs = {'class': 'lister-item-year text-muted unbold'})
        Release_Year.append(re.sub("[\D]","",release_year.get_text()))
        
        # Inner link
        link= b.find('a', href=True)
        Link.append(link['href'])
        
        # Maturity Rating 
        maturity_rating = a.find('span', {'class': 'certificate'})
        Maturity_Rating.append(maturity_rating.text if maturity_rating else 'No data')
        
        # Runtime of the movie
        runtime = a.find('span', {'class': 'runtime'})
        Runtime.append(runtime.get_text().split(' ')[0] if runtime else 'No data')
        
        # Gerne of the movie
        b = a.find('p', {'class': 'text-muted'})
        genre = b.find('span', attrs = {'class': 'genre'})
        Genre.append(re.sub('\s','',genre.get_text()) if gerne else 'No data')
        
        # html block where imdb rating and metascore is present
        b = a.find('div', attrs={'class':'ratings-bar'})
        
        # Imdb rating
        imdb_rating = b.find('strong')
        Imdb_Rating.append(imdb_rating.get_text() if imdb_rating else 'No data')
        
        # Metascore
        metascore = b.find('span', class_='metascore')
        Metascore.append(metascore.get_text().replace(' ','') if metascore else 'No data')
        
        # Number of votes given
        vote = a.find('span', attrs = {'name': 'nv'})
        Vote_Count.append(vote.get_text().replace(',','') if vote else 'No data')
        

# Iterating over the inside links that we fetched         
for link in Link:        
    driver.get('https://www.imdb.com{}'.format(link))
    
    sleep(np.random.randint(3,6,1))
    
    content = driver.page_source
    soup = BeautifulSoup(content)
        
    # Block for fetching uppar page info
    block = soup.find('div', {'class':'plot_summary_wrapper'})
    a = 0
    
    # For Director and Lead_Cast
    for i in soup.find_all('div', {'class':'credit_summary_item'}):
        
        # Exception handling to handle any none type data present in the list
        try:
            
            # Director
            if i.find('h4').get_text() == 'Director:':
                Director.append(i.find('a').get_text())
                a = a+1
            
            # Directors(if there are more than 1)
            elif i.find('h4').get_text() == 'Directors:':
                director=[]
                for b in i.find_all('a'):
                    director.append(b.get_text())
                Director.append(', '.join(director))
                a = a+1
            
            # Lead Cast
            elif i.find('h4').get_text() == 'Stars:':
                lead=[]
                for b in i.find_all('a')[0:3]:
                    lead.append(b.get_text())
                Lead_Cast.append(', '.join(lead))
                a = a+2
            else:
                None
        
        except:
            None
    
    # If there is no value present in Director or Lead then No data will get apended
    if a == 0:
        Director.append('No data')
        Lead_Cast.append('No data')
    elif a == 1:
        Lead_Cast.append('No data')
    elif a == 2:
        Director.append('No data')
    else:
        pass
    
    
    # For User_review and Critic_review count
    line = soup.find('div', {'class': 'titleReviewBarItem titleReviewbarItemBorder'})
    
    # User review
    User_Review.append(re.sub('[^\d]','',line.find_all('a')[0].get_text()) if len(line.find_all('a')) > 0 else 'No data')
    
    # Critic_Review
    Critic_Review.append(re.sub('[^\d]', '',line.find_all('a')[1].get_text()) if len(line.find_all('a')) > 1 else 'No data')
    
   
    # Block for fetching bottom page info
    block = soup.find('div', {'class':'article', 'id':'titleDetails'})
    a = 0
    
    
    for i in block.find_all('div', {'class':'txt-block'}):
        
        # Exception handling to handle any none type data present in the list
        try:
            
            # Country in which it was made
            if i.find('h4').get_text() == 'Country:':
                Country.append(i.find('a').get_text())
                a=a+1
            
            # Language released  
            elif i.find('h4').get_text() == 'Language:':
                Language.append(i.find('a').get_text())
                a=a+2
            
            # Budget of the movie
            elif i.find('h4').get_text() == 'Budget:':
                Budget.append(i.get_text().split('\n')[1].split(':')[1])
                a=a+3
                
            # Gross USA
            elif i.find('h4').get_text() == 'Gross USA:':
                Gross_USA.append(i.get_text().split('$')[1].split('\n')[0].replace(',','').split(' ')[0])
                a=a+4
                
            #Total Gross    
            elif i.find('h4').get_text() == 'Cumulative Worldwide Gross:':
                Total_Gross.append(i.get_text().split('$')[1].split('\n')[0].replace(',','').split(' ')[0])
                a=a+5
            else:
                pass
        
        except:
            None
     
    # If there is no value present in in above tags then No data will get apended
    if a==3:
        Budget.append('No data')
        Gross_USA.append('No data')
        Total_Gross.append('No data')
    elif a==6:
        Gross_USA.append('No data')
        Total_Gross.append('No data')
    elif a==7:
        Budget.append('No data')
        Total_Gross.append('No data')
    elif a==8:
        Budget.append('No data')
        Gross_USA.append('No data')
    elif a==10:
        Total_Gross.append('No data')
    elif a==11:
        Gross_USA.append('No data')
    elif a==12:
        Budget.append('No data')
    else:
        pass
    
        
       

In [533]:
# Checking length of each list
print(len(Rank), len(Movie_Name), len(Release_Year), len(Maturity_Rating), len(Runtime),len(Genre),len(Imdb_Rating), len(Metascore), len(Vote_Count),len(Link), len(Director), len(Lead_Cast), len(User_Review), len(Critic_Review), len(Country), len(Language), len(Budget), len(Gross_USA), len(Total_Gross))

5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083 5083


In [538]:
# Creating dictionary
Imdb_dict = {'Rank' : Rank,
             'Movie_Name': Movie_Name,
             'Release_Year': Release_Year,
             'Maturity_Rating': Maturity_Rating,
             'Runtime': Runtime,
             'Genre': Genre,
             'Imdb_Rating': Imdb_Rating,
             'Metascore': Metascore,
             'Vote_Count': Vote_Count,
             'Director': Director,
             'Lead_Cast': Lead_Cast,
             'User_Review': User_Review,
             'Critic_Review': Critic_Review,
             'Country': Country,
             'Language': Language,
             'Budget': Budget,
             'Gross_USA': Gross_USA,
             'Total_Gross':Total_Gross}

In [543]:
# Creating datagrame of the dictionary
Imdb = pd.DataFrame(Imdb_dict)


In [540]:
# Shape of the dataframe
Imdb.shape

(5083, 18)

In [544]:
# Head of the dataframe
Imdb.head()

,Rank,Movie_Name,Release_Year,Maturity_Rating,Runtime,Genre,Imdb_Rating,Metascore,Vote_Count,Director,Lead_Cast,User_Review,Critic_Review,Country,Language,Budget,Gross_USA,Total_Gross
0,1,Hababam Sinifi,1975,No data,87,"Comedy,Drama",9.3,No data,36161,Ertem Egilmez,"Kemal Sunal, Münir Özkul, Halit Akçatepe",62,1,Turkey,Turkish,No data,No data,No data
1,2,The Shawshank Redemption,1994,R,142,Drama,9.3,80,2269746,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton",8132,163,USA,English,"$25,000,000",28699976,28815245
2,3,The Godfather,1972,R,175,"Crime,Drama",9.2,100,1565602,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan",3932,253,USA,English,"$6,000,000",134966411,246120974
3,4,Dil Bechara,2020,12+,101,"Comedy,Drama,Romance",9.1,No data,93198,Mukesh Chhabra,"Sushant Singh Rajput, Sanjana Sanghi, Sahil Vaid",7395,28,India,Hindi,No data,No data,No data
4,5,The Dark Knight,2008,PG-13,152,"Action,Crime,Drama",9.0,84,2232889,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart",6898,420,USA,English,"$185,000,000",535234033,1037989858


In [545]:
# Tail of the dataframe
Imdb.tail()

,Rank,Movie_Name,Release_Year,Maturity_Rating,Runtime,Genre,Imdb_Rating,Metascore,Vote_Count,Director,Lead_Cast,User_Review,Critic_Review,Country,Language,Budget,Gross_USA,Total_Gross
5078,"5,079",Superbabies: Baby Geniuses 2,2004,PG,88,"Comedy,Family,Sci-Fi",1.9,9,29505,Bob Clark,"Jon Voight, Scott Baio, Vanessa Angel",130,37,Germany,English,"$20,000,000",9219388,9448644
5079,"5,080",Manos: The Hands of Fate,1966,Not Rated,70,Horror,1.9,No data,34424,Harold P. Warren,"Tom Neyman, John Reynolds, Diane Adelson",709,111,USA,English,"$19,000",No data,No data
5080,"5,081",Reis,2017,No data,108,"Biography,Drama",1.4,No data,72140,Hüdaverdi Yavuz,"Orhan Aydin, Özlem Balci, Volkan Basaran",45,14,Turkey,Turkish,"$8,000,000",No data,538551
5081,"5,082",Code Name: K.O.Z.,2015,Not Rated,114,"Crime,Mystery",1.3,No data,27073,Celal Çimen,"Cem Kurtoglu, Hakan Ural, Hazim Körmükcü",73,3,Turkey,Turkish,No data,No data,1186730
5082,"5,083",Cumali Ceber,2017,No data,100,Comedy,1.0,No data,37561,Gökhan Gök,"Halil Söyletmez, Doga Konakoglu, Emre Keskin",41,No data,Turkey,Turkish,"TRL10,000",No data,1415773


In [542]:
# Exporting it in csv format
Imdb.to_csv('C:\\Users\\Hp\\Desktop\\Analytics Vidhya Content(Offline Classes)\\Web scrapping\\imdb data.csv',index=False)